# Build an Agent
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent
The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [46]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

# Create the agent
memory = MemorySaver()
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

# Use the agent with the Tavily tool
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hi Bob! Nice to meet you. Is there anything I can help you with today?
================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (db4d66ce-c332-4adb-8d93-66380cf07a82)
 Call ID: db4d66ce-c332-4adb-8d93-66380cf07a82
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "March 2025 Weather History in San Francisco California, United ...", "url": "https://weatherspark.com/h/m/557/2025/3/Historical-Weather-in-March-2025-in-San-Francisco-California-United-States", "content": ",0l6,0l5,0l6,176l5,80l6,176l6,0l5,-80l

## Setup


### Installation
To install LangChain run:

In [ ]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
!pip install -qU "langchain[google-genai]"

### LangSmith
Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

### Tavily
We will be using Tavily (a search engine) as a tool. In order to use it, you will need to get and set an API key:

### Using Language Models
We will learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably. We will be using Gemini 2.0 Flash here.

### Entering the credentials

In [17]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass('LANGSMITH: ')
os.environ["TAVILY_API_KEY"] = getpass.getpass('TAVILY: ')
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

LANGSMITH: ··········
TRAVELY: ··········
Enter API key for Google Gemini: ··········


··········


## Define tools
We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [38]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Thursday, March 6, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/march/2025-03-06', 'content': 'San Francisco, California Weather: Thursday, March 6, 2025. Partly sunny weather with scattered clouds and occasional rain showers.', 'score': 0.94536424}, {'title': 'Weather in San Francisco in March 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/march-2025/', 'content': 'Weather in San Francisco in March 2025 San Francisco Weather Forecast for March 2025 is based on statistical data. March +50° +50° +48° +50° +52° +46° +46° +46° +46° +46° +48° +54° +46° +46° +48° +50° +52° +46° +45° +46° +48° +50° +50° +50° +59° +55° +55° +54° +48° +50° +54° Average weather in March 2025 Extended weather forecast in San Francisco Weather in large and nearby cities Weather in Washington, D.C.+55° Sacramento+82° Pleasanton+73° Redwood City+68° San Leandro+64° San Mateo+64° San Rafael+66° San Ramon

## Initialize the Language Model


In [45]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

You can call the language model by passing in a list of messages. By default, the response is a content string.

In [44]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [20]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.

In [21]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! How can I help you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [27]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '54366354-652a-47b7-be14-e131de85e51c', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the model, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [40]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent
We can now run the agent with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:



In [41]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='8db5c8dc-c3f7-4f10-a7f4-c3bde9d61384'),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8875c1b3-d3b1-40bc-9220-de4de381d41e-0', usage_metadata={'input_tokens': 51, 'output_tokens': 11, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}})]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r/fe129b40-7c0a-4e9e-82e3-1c19b00d04f0)

Let's now try it out on an example where it should be invoking the tool (Tavily):

In [42]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='b94a2fa8-9b2c-4638-9f13-13188c8ff852'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a27d19b2-8ff2-4f6e-ac5d-ba767da9e864-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'cd270d5c-1def-4049-9db4-9f5319254319', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 13, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "March 2025 Weather History in San Francisco California, United ...", "url": "https://weatherspark.com/h/m/557/2025/3/Historical-Weather-in-March-2025-in-San-Francisco-Califo

We can check out the LangSmith trace to make sure it's calling the search tool effectively.

## Streaming Messages
We've seen how the agent can be called with .invoke to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [39]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'checkpoint_ns', 'checkpoint_id']

## Streaming tokens
In addition to streaming back messages, it is also useful to stream back tokens. We can do this by specifying stream_mode="messages".

Below we use` message.text()`, which requires langchain-core>=0.3.37.


In [32]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I am sorry,| I cannot provide specific real-time weather conditions for San Francisco using the available tools.| The search results provide historical weather data for March 2025, which| is not what you are asking for.
|

## Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [35]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c808ad7f-c424-45fe-a23b-f69db5237b9e-0', usage_metadata={'input_tokens': 53, 'output_tokens': 11, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})]}}
----


Let's see if it does record that I'm Bob:

In [36]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a050b1c1-1cab-438c-8027-dfb977dd0878-0', usage_metadata={'input_tokens': 67, 'output_tokens': 6, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})]}}
----


If you want to start a new conversation, all you have to do is change the `thread_id` used. In the execution below, it will not know my name.

In [37]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I do not have access to personal information, so I don't know your name.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e1509b4d-8d10-458c-8cba-90e08536eff8-0', usage_metadata={'input_tokens': 53, 'output_tokens': 19, 'total_tokens': 72, 'input_token_details': {'cache_read': 0}})]}}
----


## Conclusion
That's a wrap! In this quick start we covered how to create a simple agent. We've then shown how to stream back a response - not only with the intermediate steps, but also tokens! We've also added in memory so you can have a conversation with them. Agents are a complex topic with lots to learn!

For more information on Agents, please check out the [LangGraph ](https://python.langchain.com/docs/concepts/architecture/#langgraph)documentation. This has it's own set of concepts, tutorials, and how-to guides.
